# Exercise 1
Read and parse the chemical reactions `.xml` input file `rxns.xml`.


1. Collect the species into a species list.  My output is `['H', 'O', 'OH', 'H2', 'O2']`.
    
    Some notes and hints:
    * **Hint:**  For this `.xml` format you should have a loop over the `phase` element.
    * **Hint:** You can use the `find()` method to get the species array.

2. Calculate and print out the Arrhenius reaction rate coefficients using $R = 8.314$ and $T = 1500$.

    Some notes and hints:
    * **Hint:**  For this `.xml` format you should have loops over the `reactionData` element, the `reaction` element, the `rateCoeff` element, and the `Arrhenius` element using the `findall()` method discussed in lecture.
    * **Hint:** You can use the `find()` method to get the reaction rate coefficients.
    * My solution is:
    
        `k for reaction01 =    6.8678391864294477e+05
    k for reaction02 =    2.3105559199959813e+06`

In [1]:
###############################################################################
# Part 1
###############################################################################

import xml.etree.ElementTree as ET

tree = ET.parse('rxns.xml')
root = tree.getroot()
species = []
for species_element in root.find('phase'):
    new_species = species_element.text.strip().split()
    species.extend(new_species)
    
print(species)

['H', 'O', 'OH', 'H2', 'O2']


In [2]:
###############################################################################
# Part 2
# arrhenius functions copied over from prior HW.
###############################################################################

import math

def arrhenius (A: float, E: float, T: float, R: float = 8.314) -> float:
    """
    Args:
        A: Pre-exponential (frequency) factor.
            ValueError raised if not > 0.
        E: Activation energy.
        T: Temperature in Kelvins.
            ValueError raised if not >= 0.
        R: Universal gas constant.

    Returns:
        Arrhenius reaction rate coefficient.
    """
    if A <= 0:
        raise ValueError('A must be > 0.')
    if T < 0:
        raise ValueError('T must be >= 0.')

    return A * math.exp(-E / (R * T))

def modified_arrhenius (A: float, E: float, T: float, b: float, R: float = 8.314) -> float:
    """
    Args:
        A: Arrhenius prefactor (frequency factor).
            ValueError raised if not > 0.
        E: Activation energy.
        T: Temperature in Kelvins.
            ValueError raised if not >= 0.
        b: Modified Arrhenius parameter.
            Must be real number.
        R: Universal gas constant.

    Notes:
        Calculation done by multiplying the result of a call to arrhenius() with the
        modified Arrhenius parameter (T^b).

    Returns:
        Modified Arrhenius reaction rate coefficient.
    """
    if isinstance(b, complex):
        raise ValueError('b must be real number.')
    prelim = arrhenius(A, E, T, R)
    return prelim * (T**b)

In [3]:
T = 1500

# Store Arrhenius coefficients in dict with key equal to id attribute in xml.
results = {}

for reaction in root.find('reactionData').findall('reaction'):
    for rate_coeff in reaction.findall('rateCoeff'):
        for i, arrhen in enumerate(rate_coeff.findall('Arrhenius')):
            A = float(arrhen.find('A').text)
            b = float(arrhen.find('b').text)
            E = float(arrhen.find('E').text)
            results[reaction.get('id')] = arrhenius(A, E, T, b)
            
for reaction_id, k in results.items():
    print('k for {} = {}'.format(reaction_id, k))

k for reaction01 = 1.1983051695759727e+40
k for reaction02 = 7.654506824737527e-05
